<a href="https://colab.research.google.com/github/blakelobato/DS-Unit-4-Sprint-3-Deep-Learning/blob/master/module2-convolutional-neural-networks/432A_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

## *Data Science Unit 4 Sprint 3 Assignment 2*
# Convolutional Neural Networks (CNNs)

# Assignment

- <a href="#p1">Part 1:</a> Pre-Trained Model
- <a href="#p2">Part 2:</a> Custom CNN Model
- <a href="#p3">Part 3:</a> CNN with Data Augmentation


You will apply three different CNN models to a binary image classification model using Keras. Classify images of Mountains (`./data/mountain/*`) and images of forests (`./data/forest/*`). Treat mountains as the postive class (1) and the forest images as the negative (zero). 

|Mountain (+)|Forest (-)|
|---|---|
|![](https://github.com/blakelobato/DS-Unit-4-Sprint-3-Deep-Learning/blob/master/module2-convolutional-neural-networks/data/mountain/art1131.jpg?raw=1)|![](https://github.com/blakelobato/DS-Unit-4-Sprint-3-Deep-Learning/blob/master/module2-convolutional-neural-networks/data/forest/cdmc317.jpg?raw=1)|

The problem is realively difficult given that the sample is tiny: there are about 350 observations per class. This sample size might be something that you can expect with prototyping an image classification problem/solution at work. Get accustomed to evaluating several differnet possible models.

# Pre - Trained Model
<a id="p1"></a>

Load a pretrained network from Keras, [ResNet50](https://tfhub.dev/google/imagenet/resnet_v1_50/classification/1) - a 50 layer deep network trained to recognize [1000 objects](https://storage.googleapis.com/download.tensorflow.org/data/ImageNetLabels.txt). Starting usage:

```python
import numpy as np

from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions

from tensorflow.keras.layers import Dense, GlobalAveragePooling2D()
from tensorflow.keras.models import Model # This is the functional API

resnet = ResNet50(weights='imagenet', include_top=False)

```

The `include_top` parameter in `ResNet50` will remove the full connected layers from the ResNet model. The next step is to turn off the training of the ResNet layers. We want to use the learned parameters without updating them in future training passes. 

```python
for layer in resnet.layers:
    layer.trainable = False
```

Using the Keras functional API, we will need to additional additional full connected layers to our model. We we removed the top layers, we removed all preivous fully connected layers. In other words, we kept only the feature processing portions of our network. You can expert with additional layers beyond what's listed here. The `GlobalAveragePooling2D` layer functions as a really fancy flatten function by taking the average of each of the last convolutional layer outputs (which is two dimensional still). 

```python
x = res.output
x = GlobalAveragePooling2D()(x) # This layer is a really fancy flatten
x = Dense(1024, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)
model = Model(res.input, predictions)
```

Your assignment is to apply the transfer learning above to classify images of Mountains (`./data/mountain/*`) and images of forests (`./data/forest/*`). Treat mountains as the postive class (1) and the forest images as the negative (zero). 

Steps to complete assignment: 
1. Load in Image Data into numpy arrays (`X`) 
2. Create a `y` for the labels
3. Train your model with pretrained layers from resnet
4. Report your model's accuracy

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import backend as K
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Flatten, Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.applications.inception_resnet_v2 import InceptionResNetV2, preprocess_input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau


DATASET_PATH  = '/content/drive/My Drive/Colab Notebooks/Unit 4 Sprint 3 Major NN Arch/data'

In [0]:
from skimage.io import imread_collection
from skimage.transform import resize
from sklearn.model_selection import train_test_split

## Load in Data

![skimage-logo](https://scikit-image.org/_static/img/logo.png)

Check out out [`skimage`](https://scikit-image.org/) for useful functions related to processing the images. In particular checkout the documentation for `skimage.io.imread_collection` and `skimage.transform.resize`.

In [0]:
#your path 
forests = '/content/drive/My Drive/Colab Notebooks/Unit 4 Sprint 3 Major NN Arch/data/train/forest/*.jpg'
mountains = '/content/drive/My Drive/Colab Notebooks/Unit 4 Sprint 3 Major NN Arch/data/train/mountain/*.jpg'
val_forests = '/content/drive/My Drive/Colab Notebooks/Unit 4 Sprint 3 Major NN Arch/data/validation/forest/*.jpg'
val_mountains = '/content/drive/My Drive/Colab Notebooks/Unit 4 Sprint 3 Major NN Arch/data/validation/mountain/*.jpg'

In [0]:
#creating a collection with the available images
forests = imread_collection(forests).concatenate()
mountains = imread_collection(mountains).concatenate()

In [0]:
forests.shape

(268, 256, 256, 3)

In [0]:
y_0 =  np.zeros(forests.shape[0])
y_1 =  np.ones(mountains.shape[0])

In [0]:
X = np.concatenate([forests, mountains])
X = resize(X, (520,256,256,3))
y = np.concatenate([y_0, y_1])

In [0]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=.5)

In [0]:
x_train

array([[[[0.69019608, 0.54509804, 0.43137255],
         [0.74509804, 0.58823529, 0.49019608],
         [0.79215686, 0.62745098, 0.53333333],
         ...,
         [0.8745098 , 0.56862745, 0.41960784],
         [0.83529412, 0.52941176, 0.38823529],
         [0.85490196, 0.54117647, 0.40392157]],

        [[0.62745098, 0.4745098 , 0.35294118],
         [0.73333333, 0.57254902, 0.4627451 ],
         [0.78823529, 0.62352941, 0.5254902 ],
         ...,
         [0.84705882, 0.54117647, 0.39215686],
         [0.83529412, 0.52941176, 0.38039216],
         [0.83921569, 0.54509804, 0.39215686]],

        [[0.66666667, 0.49803922, 0.37254902],
         [0.68235294, 0.52156863, 0.41176471],
         [0.80784314, 0.64313725, 0.54509804],
         ...,
         [0.8627451 , 0.54901961, 0.41176471],
         [0.85882353, 0.55294118, 0.40392157],
         [0.84313725, 0.54901961, 0.39215686]],

        ...,

        [[0.07843137, 0.30196078, 0.        ],
         [0.16862745, 0.39215686, 0.08235294]

## Instatiate Model

In [0]:
x_train.shape

(260, 256, 256, 3)

In [0]:
# Create the base model from the pre-trained model ResNet
base_model = tf.keras.applications.resnet50.ResNet50(weights='imagenet', include_top=False, input_shape= (256,256,3))

for layer in base_model.layers:
    layer.trainable = False

94773248/94765736 [==============================] - 1s 0us/step


In [0]:
base_model.trainable = False

In [0]:
# Let's take a look at the base model architecture
base_model.summary()

Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 256, 256, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 262, 262, 3)  0           input_2[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 128, 128, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 128, 128, 64) 256         conv1_conv[0][0]                 
___________________________________________________________________________________________

In [0]:
DATASET_PATH  = '/content/drive/My Drive/Colab Notebooks/Unit 4 Sprint 3 Major NN Arch/data'
IMAGE_SIZE    = (255, 255)
NUM_CLASSES   = 2
BATCH_SIZE    = 5  # try reducing batch size or freeze more layers if your GPU runs out of memory

In [0]:
train_data = ImageDataGenerator(preprocessing_function=preprocess_input)

train_batches = train_data.flow_from_directory(DATASET_PATH + '/train',
                                                  target_size=IMAGE_SIZE,
                                                  class_mode='categorical',
                                                  shuffle=True,
                                                  batch_size=BATCH_SIZE)

valid_data = ImageDataGenerator(preprocessing_function=preprocess_input)



valid_batches = valid_data.flow_from_directory(DATASET_PATH + '/validation',
                                                  target_size=IMAGE_SIZE,
                                                  class_mode='categorical',
                                                  shuffle=True,
                                                  batch_size=BATCH_SIZE)

Found 520 images belonging to 2 classes.
Found 182 images belonging to 2 classes.


In [0]:
# show class indices
print('****************')
for cls, idx in train_batches.class_indices.items():
    print('Class #{} = {}'.format(idx, cls))
print('****************')

****************
Class #0 = forest
Class #1 = mountain
****************


In [0]:
base_model = tf.keras.applications.resnet50.ResNet50(weights='imagenet', include_top=False, input_shape=(IMAGE_SIZE[0],IMAGE_SIZE[1],3))

for layer in base_model.layers:
    layer.trainable = False

In [0]:
x = base_model.output
x = Flatten()(x)
output_layer = Dense(NUM_CLASSES, activation='softmax', name='softmax')(x)
network = Model(inputs=base_model.input, outputs=output_layer)

In [0]:
network.compile(optimizer=Adam(lr=0.001),
                  loss='categorical_crossentropy', metrics=['accuracy'])

In [0]:
stop = EarlyStopping(monitor='val_accuracy', mode='max', min_delta=0.01, patience=3, verbose=1)

In [0]:
network.fit(
        train_batches,
        epochs=10,
        validation_data=valid_batches,
        verbose=1,
        steps_per_epoch = train_batches.samples // BATCH_SIZE,
        callbacks=[stop],
        validation_steps = valid_batches.samples // BATCH_SIZE)

Epoch 1/10
104/104 [==============================] - 140s 1s/step - loss: 3.0188 - accuracy: 0.7538 - val_loss: 2.8799 - val_accuracy: 0.7000
Epoch 2/10
104/104 [==============================] - 137s 1s/step - loss: 1.3443 - accuracy: 0.8500 - val_loss: 0.6967 - val_accuracy: 0.9167
Epoch 3/10
104/104 [==============================] - 135s 1s/step - loss: 0.8530 - accuracy: 0.9115 - val_loss: 3.7993 - val_accuracy: 0.7222
Epoch 4/10
104/104 [==============================] - 136s 1s/step - loss: 0.1349 - accuracy: 0.9827 - val_loss: 0.6439 - val_accuracy: 0.8944
Epoch 5/10
104/104 [==============================] - 136s 1s/step - loss: 0.0662 - accuracy: 0.9865 - val_loss: 1.4625 - val_accuracy: 0.8278
Epoch 00005: early stopping


# Custom CNN Model

In this step, write and train your own convolutional neural network using Keras. You can use any architecture that suits you as long as it has at least one convolutional and one pooling layer at the beginning of the network - you can add more if you want. 

In [0]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=.5)

In [0]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten

# Setup Architecture
model = Sequential()
model.add(Conv2D(32, (3,3), activation='relu', input_shape=(256,256,3)))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(64, (3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(64, (3,3), activation='relu'))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(1, activation='softmax'))

# Compile Model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 254, 254, 32)      896       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 127, 127, 32)      0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 125, 125, 64)      18496     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 62, 62, 64)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 60, 60, 64)        36928     
_________________________________________________________________
flatten_1 (Flatten)          (None, 230400)            0         
_________________________________________________________________
dense_2 (Dense)              (None, 64)               

In [0]:
# Print summary
model.fit(train_batches,
          epochs=10,
          validation_data=valid_batches)

Epoch 1/10


InvalidArgumentError: ignored

In [0]:
# Normalize from 0-1
x_train = x_train.astype('float32') / 260.
x_test = x_test.astype('float32') / 260.

# Reshaping the array to 4-dims so that it can work with the Tensorflow.Keras API
#X_train = X_train.reshape(X_train.shape[0], 28, 28, 1)
#X_test = X_test.reshape(X_test.shape[0], 28, 28, 1)

# Setup Architecture
model = Sequential()
model.add(Conv2D(32, (3,3), activation='relu', input_shape=(256,256,3)))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(64, (3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(64, (3,3), activation='relu'))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(10, activation='softmax'))

# Compile Model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Print summary
model.fit(x_train, y_train, 
          epochs=5, 
          validation_data=(x_test, y_test))

Epoch 1/5
9/9 [==============================] - 28s 3s/step - loss: 1.3718 - accuracy: 0.4769 - val_loss: 0.8538 - val_accuracy: 0.5231
Epoch 2/5
9/9 [==============================] - 28s 3s/step - loss: 0.7843 - accuracy: 0.5154 - val_loss: 0.7391 - val_accuracy: 0.5231
Epoch 3/5
9/9 [==============================] - 28s 3s/step - loss: 0.7570 - accuracy: 0.5000 - val_loss: 0.7241 - val_accuracy: 0.4769
Epoch 4/5
9/9 [==============================] - 28s 3s/step - loss: 0.7040 - accuracy: 0.5231 - val_loss: 0.7331 - val_accuracy: 0.5231
Epoch 5/5
9/9 [==============================] - 28s 3s/step - loss: 0.7128 - accuracy: 0.5231 - val_loss: 0.7279 - val_accuracy: 0.4769


# Custom CNN Model with Image Manipulations
## *This a stretch goal, and it's relatively difficult*

To simulate an increase in a sample of image, you can apply image manipulation techniques: cropping, rotation, stretching, etc. Luckily Keras has some handy functions for us to apply these techniques to our mountain and forest example. Check out these resources to help you get started: 

1. [Keras `ImageGenerator` Class](https://keras.io/preprocessing/image/#imagedatagenerator-class)
2. [Building a powerful image classifier with very little data](https://blog.keras.io/building-powerful-image-classification-models-using-very-little-data.html)
 

In [0]:
# State Code for Image Manipulation Here

# Resources and Stretch Goals

Stretch goals
- Enhance your code to use classes/functions and accept terms to search and classes to look for in recognizing the downloaded images (e.g. download images of parties, recognize all that contain balloons)
- Check out [other available pretrained networks](https://tfhub.dev), try some and compare
- Image recognition/classification is somewhat solved, but *relationships* between entities and describing an image is not - check out some of the extended resources (e.g. [Visual Genome](https://visualgenome.org/)) on the topic
- Transfer learning - using images you source yourself, [retrain a classifier](https://www.tensorflow.org/hub/tutorials/image_retraining) with a new category
- (Not CNN related) Use [piexif](https://pypi.org/project/piexif/) to check out the metadata of images passed in to your system - see if they're from a national park! (Note - many images lack GPS metadata, so this won't work in most cases, but still cool)

Resources
- [Deep Residual Learning for Image Recognition](https://arxiv.org/abs/1512.03385) - influential paper (introduced ResNet)
- [YOLO: Real-Time Object Detection](https://pjreddie.com/darknet/yolo/) - an influential convolution based object detection system, focused on inference speed (for applications to e.g. self driving vehicles)
- [R-CNN, Fast R-CNN, Faster R-CNN, YOLO](https://towardsdatascience.com/r-cnn-fast-r-cnn-faster-r-cnn-yolo-object-detection-algorithms-36d53571365e) - comparison of object detection systems
- [Common Objects in Context](http://cocodataset.org/) - a large-scale object detection, segmentation, and captioning dataset
- [Visual Genome](https://visualgenome.org/) - a dataset, a knowledge base, an ongoing effort to connect structured image concepts to language